In [ ]:
"""
Author: Tharnier O. Puel
Last Modified: Feb 2025

Description: 
    Utilizes PyTorch.optim to perform a parameterized fitting of the Hamiltonian for the trivalent lanthanide ions.

License: 
    All rights reserved to proprietary.

Contact: 
    Email  : tharnier@me.com
    Website: https://topuel.wordpress.com
    GitHub : https://github.com/Tharnier
    

Editable Lines:
    - Cell title [# Import Hamiltonian matrices from files]:
        Adjust the directory path:
        [directory = 'symbolic_Hamiltonian_Er/']
    
    - Cell title [# Prepare target eigenvalues]:
        Adjust the file path:
        [with open("Experimental_energies/Er-spectrum.m", "r") as file:]
    
    - Cell title [# Define method and seed for PyTorch optimization]:
        Modify trial parameter declaration lines:
        [param = pt.tensor(1234., requires_grad=True)]
        Update fitting_params = {dictionary} to include all declared trial parameters.
        
        Adjust lines:
        [fixed_params = {dictionary}]
        [ratios = {dictionary}]
    
    - Cell title [# PyTorch optimization loop]
        Adjust line:
        [num_iterations = 100]

    - Cell title [# Save results to files]:
        Modify directory and file name for results storage:
        [directory = 'fitting-trials/']
        [file_name = 'Er_PyTorch']

Notes: 
    - Ensure all paths are valid before running the notebook.
    - Review editable sections before executing cells to avoid runtime errors.
"""

In [1]:
# ----------------
# Import libraries 
# ----------------

import os      as os # Handle file's paths and names
import torch   as pt # Tensor algebra, used for matrix diagonalization
import time    as tm # Keep track of time during iterations
import pandas  as pd # Handle mixed data types and used to save .CSV files

In [ ]:
# --------------------------------------
# Import Hamiltonian matrices from files
# --------------------------------------

# Directory containing symbolic Hamiltonian files
directory = 'symbolic_Hamiltonian_Er/'

# Store file contents as expressions
file_expressions = {}

# Process files (ended with .m) and store content in the dictionary
for file_name in os.listdir(directory):
    if file_name.endswith('.m'):
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'r') as file:
            # Read file and strip unnecessary spaces/newlines
            content = file.read().strip().replace("\n","")
            # Remove headers created by Mathematica
            content = content.replace("(* Created with the Wolfram Language : www.wolfram.com *)", "").replace("(* ::Package:: *)", "")
            # Create dictionary key
            key = file_name.replace("Er_matrix_for_python_","").replace(".m","")
            # Update dictionary
            file_expressions[key] = content

# Output the dictionary keys
print( file_expressions.keys() )

In [3]:
# -----------------------------------------------------
# Function to convert Mathematica expressions to Python
# ----------------------------------------------------- 

def convert_mathematica_to_Python(matrix_string, type='pytorch'):
    """
    Converts a Mathematica-style matrix string to a PyTorch tensor or Pandas data.

    Args:
        matrix_string (str): The Mathematica-style matrix string.

    Returns:
        pt.tensor or pd.DataFrame: The converted matrix in either PyTorch of Pandas format.
    """
    # Replace Mathematica-specific syntax with Python-compatible syntax
    python_style = (
        matrix_string
        .replace('Sqrt[', 'pt.tensor(')  # Convert Mathematica Sqrt[...] to PyTorch pt.tensor(...).sqrt()
        .replace(']',').sqrt()')         # Convert Mathematica Sqrt[...] to PyTorch pt.tensor(...).sqrt()
        .replace('{', '[')               # Convert Mathematica braces to Python brackets
        .replace('}', ']')               # Convert Mathematica braces to Python brackets
        .replace('I', '1j')              # Convert Mathematica complex I to Python 1j
        .replace('*^', 'e')              # Convert Mathematica scientific notation
    )
    
    # Parse and execute expression provided as a string
    matrix_list = eval(python_style, {"pt" : pt, "j": 1j})

    # Convert to Pandas data or PyTorch tensor
    if type == 'pandas':
        output = pd.DataFrame(matrix_list)
    elif type == 'pytorch':
        output = pt.tensor(matrix_list, dtype=pt.complex128)
    else:
        print('From convert_mathematica_to_Python(): invalid option (type)')

    # Convert to PyTorch tensor
    return output

In [ ]:
# -------------------------
# Prepare target eigenvalues
# -------------------------

# Open and read eigenvalues file
with open("Experimental_energies/Er-spectrum.m", "r") as file:
    # Read file and strip unnecessary spaces/newlines
    content = file.read().strip().replace("\n","")
    # Remove headers created by Mathematica
    file_expressions_target_eigenvalues = content.replace("(* Created with the Wolfram Language : www.wolfram.com *)", "").replace("(* ::Package:: *)", "")

# Define the Mathematica-like expression as a string
target_eigenvalues = convert_mathematica_to_Python( file_expressions_target_eigenvalues, type='pandas' )

# Find numerical values
mask = target_eigenvalues.applymap(lambda value: isinstance(value, float))
# Filter target eigenvalues to numeric values only
numeric_target_eigenvalues = target_eigenvalues[mask].dropna().astype(float)

# Convert indices and values to PyTorch tensors
target_index_set          = pt.tensor(numeric_target_eigenvalues.index , dtype=pt.long).flatten()
target_eigenvalues_tensor = pt.tensor(numeric_target_eigenvalues.values, dtype=pt.float64).flatten()

# Shift and sort energies to start at zero
target_eigenvalues_tensor, idx = pt.sort( target_eigenvalues_tensor - pt.min(target_eigenvalues_tensor) )

# Display imported values
print( target_eigenvalues_tensor )
print( target_index_set )

In [ ]:
# -------------------------------------------
# Convert Mathematica expressions into Python
# -------------------------------------------

# Apply Mathematica to Python convertion to the files expressions
matrices_dictionary = {key: convert_mathematica_to_Python(value, type='pytorch') for key, value in file_expressions.items()}

# Output the dictionary keys
print( matrices_dictionary.keys() )

In [6]:
# -------------------------------------------------
# Function to assemble the parametrized Hamiltonian
# -------------------------------------------------

def matrix_Hamiltonian(fitting_params, fixed_params = {}, ratios = {}):
    """
    Add matrices weighted by their coefficients (parameters).
    Notice that it will only sum over the provided parameters.

    Args:
        fitting_dictionary: Dictionary parameters to be fitted.
        fixed_params: Dictionary with parameters left out from the fitting.
        ratios: { 'P4/P2': value, 'P6/P2': value, 'M2/M0': value, 'M4/M0': value }

    Returns:
        pt.tensor: The assembled matrix as PyTorch tensor.
    """
    # CHECK if the same param appears in both dictionaries
    common_keys = fitting_params.keys() & fixed_params.keys() | ratios.keys() & fitting_params.keys() | ratios.keys() & fixed_params.keys()
    if common_keys:
        raise SystemExit(f'From matrix_Hamiltonian(). The following params appear in multiple dictionaries: {common_keys}')

    # CHECK if keys provided in fitting_params and fixed_params exist in Hamiltonian (matrices_dictionary)
    if not common_keys.issubset(matrices_dictionary.keys()):
        missing_keys = common_keys - matrices_dictionary.keys()
        raise SystemExit(f'From matrix_Hamiltonian(). The following parameter keys are missing in the Hamiltonian: {missing_keys}')

    # Multiply matrices by their corresponding parameters and return their sum (only over the provided parameters)
    numerical_matrix = sum(fitting_params[key] * matrices_dictionary[key] for key in fitting_params)
    if fixed_params:
        numerical_matrix += sum(fixed_params[key] * matrices_dictionary[key] for key in fixed_params)

    # Add P4 and P6 parameters as ratios to P2
    if {'P4/P2'}.issubset(ratios.keys()) or {'P6/P2'}.issubset(ratios.keys()):
        if   not {'P4/P2','P6/P2'}.issubset(ratios.keys()):
            raise SystemExit('From matrix_Hamiltonian(). Both keys \'P4/P2\' and \'P6/P2\' must be defined')
        elif not {'P2'}.issubset(fitting_params.keys()):
            raise SystemExit('From matrix_Hamiltonian(). The ratios option only accepts P2 as a fitting parameter')
        elif {'P4'} & fitting_params.keys() | {'P4'} & fixed_params.keys() | {'P6'} & fitting_params.keys() | {'P6'} & fixed_params.keys():
            raise SystemExit('From matrix_Hamiltonian(). Parameters P4 and P6 cannot be defined if using P4/P2 and P6/P2 ratios')
        elif not {'P4','P6'}.issubset(matrices_dictionary.keys()):
            raise SystemExit('From matrix_Hamiltonian(). One or more of the following parameter keys are missing in the Hamiltonian: P4 and P6')
        else:
            numerical_matrix += fitting_params['P2']*ratios['P4/P2']*matrices_dictionary['P4'] + fitting_params['P2']*ratios['P6/P2']*matrices_dictionary['P6']
    
    # Add M2 and M4 parameters as ratios to M0
    if {'M2/M0'}.issubset(ratios.keys()) or {'M4/M0'}.issubset(ratios.keys()):
        if   not {'M2/M0','M4/M0'}.issubset(ratios.keys()):
            raise SystemExit('From matrix_Hamiltonian(). Both keys \'M2/M0\' and \'M4/M0\' must be defined')
        elif   not {'M0'}.issubset(fitting_params.keys()):
            raise SystemExit('From matrix_Hamiltonian(). The ratios option only accepts M0 as a fitting parameter')
        elif {'M2'} & fitting_params.keys() | {'M2'} & fixed_params.keys() | {'M4'} & fitting_params.keys() | {'M4'} & fixed_params.keys():
            raise SystemExit('From matrix_Hamiltonian(). Parameters M2 and M4 cannot be defined if using M2/M0 and M4/M0 ratios')
        elif not {'M2','M4'}.issubset(matrices_dictionary.keys()):
            raise SystemExit('From matrix_Hamiltonian(). One or more of the following parameter keys are missing in the Hamiltonian: M2 and M4')
        else:
            numerical_matrix += fitting_params['M0']*ratios['M2/M0']*matrices_dictionary['M2'] + fitting_params['M0']*ratios['M4/M0']*matrices_dictionary['M4']

    return numerical_matrix

In [7]:
# ----------------------------------------
# Objective function for PyTorch optimizer
# ----------------------------------------

def objective(fitting_params, fixed_params = {}, ratios = {}):
    """
    Computes the objective value for optimizing a parametrized Hamiltonian

    Args:
        params_dictionary: Dictionary with parameters and their values.

    Returns:
        float: The normalized objective value, representing the relative difference between computed and target eigenvalues.
    """
    # Assemble Hamiltonian matrix
    A_tensor = matrix_Hamiltonian(fitting_params, fixed_params, ratios)
    
    # Next line is optional, if one needs to use CUDA
    #A_tensor = A_tensor.to('cuda')
    
    # Use PyTorch for computing the eigenvalues
    eigenvalues = pt.linalg.eigvals(A_tensor).real.cpu()  # .cpu() is only really necessary if using CUDA
    
    # Sort and shift
    eigenvalues, idx = pt.sort( eigenvalues - pt.min(eigenvalues) )

    # Filter eigenvalues according to target values
    eigenvalues_filtered = eigenvalues[target_index_set]

    # Compute the absolute difference
    abs_diff = pt.abs(eigenvalues_filtered - target_eigenvalues_tensor)

    # Compute the denominator
    denom    = len(target_index_set) - len(fitting_params)

    # Compute the objective function (as in see Carnall et al. J.Chem.Phys. 90, 3443 (1989))
    objective_value = pt.sqrt( pt.sum( abs_diff**2 ) / denom )

    # Return objective value
    return objective_value

In [8]:
# -----------------------------------------------
# Define method and seed for PyTorch optimization
# -----------------------------------------------
    
# Suggested seed values for parameters (see Carnall et al. J.Chem.Phys. 90, 3443 (1989))
f2      = pt.tensor(97483.  , requires_grad=True)
f4      = pt.tensor(67904.  , requires_grad=True)
f6      = pt.tensor(54010.  , requires_grad=True)
zeta    = pt.tensor(2376.   , requires_grad=True)
alpha   = pt.tensor(17.79   , requires_grad=True)
beta    = pt.tensor(-582.1  , requires_grad=True)
t3      = pt.tensor(43.     , requires_grad=True)
t4      = pt.tensor(73.     , requires_grad=True)
t6      = pt.tensor(-271.   , requires_grad=True)
t7      = pt.tensor(308.    , requires_grad=True)
t8      = pt.tensor(299.    , requires_grad=True)
m0      = pt.tensor(3.86    , requires_grad=True)
p2      = pt.tensor(594.    , requires_grad=True)
b02     = pt.tensor(-238.   , requires_grad=True)
b04     = pt.tensor(453.    , requires_grad=True)
b06     = pt.tensor(373.    , requires_grad=True)
b22     = pt.tensor(-91.    , requires_grad=True)
b24     = pt.tensor(308.    , requires_grad=True)
b44     = pt.tensor(417.    , requires_grad=True)
b26     = pt.tensor(-489.   , requires_grad=True)
b46     = pt.tensor(-240.   , requires_grad=True)
b66     = pt.tensor(-536.   , requires_grad=True)

# Parameters dictionary
fitting_params = { 'F2': f2, 'F4': f4, 'F6': f6, 'Zeta': zeta, 'Alpha': alpha, 'Beta': beta, \
                'T3': t3, 'T4': t4, 'T6': t6, 'T7': t7, 'T8': t8, 'P2': p2, 'M0': m0, \
                'B02': b02, 'B04': b04, 'B06': b06, 'B22': b22, 'B24': b24, 'B44': b44, \
                'B26': b26, 'B46': b46, 'B66': b66 }

# Fixed parameters
fixed_params   = { 'Gamma': 1800., 'T2': 400. }
ratios         = { 'M2/M0': 0.56, 'M4/M0': 0.31, 'P4/P2': 0.5, 'P6/P2': 0.1 }

# Keep initial parameters
numeric_initial_params = {key: round(value.detach().item(),10) for (key, value) in fitting_params.items()} | fixed_params | ratios

# Choose one of the following optimizer methods
optimizer = pt.optim.Adam( fitting_params.values(), lr=0.1)
#optimizer = pt.optim.RMSprop( fitting_params.values(), lr=0.01, alpha=0.99) 
#optimizer = pt.optim.SGD( fitting_params.values(), lr=0.01, momentum=0.9) 
#optimizer = pt.optim.Adagrad( fitting_params.values(), lr=0.01) 
#optimizer = pt.optim.LBFGS( fitting_params.values(), lr=0.01) 
#optimizer = pt.optim.AdamW( fitting_params.values(), lr=0.001, weight_decay=0.01)

In [ ]:
# -------------------------
# PyTorch optimization loop
# -------------------------

# Number of iterations
num_iterations = 100

# Keep track of the best iteration
best_loss, best_iteration, best_fitting_params = float('inf'), -1, None

# List to store output
output_data = []

for iteration in range(num_iterations):
    
    def closure():
       # Reset gradients
       optimizer.zero_grad()
       
       # Build matrix, compute eigenvalues, return loss (comparison to target values)
       current_loss = objective(fitting_params, fixed_params, ratios)
       #print(loss.requires_grad)  # Should return True in order to use .backward()
       
       # Backpropagate
       current_loss.backward()
       
       return current_loss
    
    # Update parameters
    loss = optimizer.step(closure)

    # Update the best iteration
    if loss.item() < best_loss:
        best_loss, best_iteration = loss.item(), iteration
        best_fitting_params = fitting_params

    # Convert 'params' list from tensor to numerical form
    numeric_params = {key: round(value.detach().item(),10) for (key, value) in fitting_params.items()} | fixed_params | ratios

    # Keep iteration infos
    iteration_output = {
        "Time": tm.strftime("%H:%M:%S",tm.localtime(tm.time())),
        "Iteration": iteration,
        "Loss": round(loss.item(),10),
        "Parameters": numeric_params,
        "Best iteration": best_iteration,
        "Lowest loss": round(best_loss, 10),
    }
    output_data.append(iteration_output)

    # Print progress every iteration
    print(iteration_output)

In [ ]:
# --------------------------------------
# After convergence, compute the Hessian
# --------------------------------------

# Recompute loss for the last parameters
loss = current_loss = objective(best_fitting_params, fixed_params, ratios)   

# Compute the Hessian
hessian = []
for param in fitting_params.values():
    # First-order gradient for the current parameter
    grad = pt.autograd.grad(loss, param, create_graph=True)[0].view(-1)
    for g in grad:
        # Second-order gradients (Hessian rows)
        hessian_row = pt.autograd.grad(g, fitting_params.values(), retain_graph=True)
        hessian.append(pt.cat([h.view(-1) for h in hessian_row]))

# Stack all rows to form the Hessian matrix
hessian = pt.stack(hessian)

# Compute the inverse of the Hessian (Covariance matrix)
hessian_inverse = pt.linalg.inv(hessian)

# Extract standard deviations (square root of diagonal elements)
uncertainties = pt.sqrt(pt.diag(hessian_inverse))

# Create a dictionary with parameter names and their uncertainties
uncertainties_dict = {key: value.detach().item() for key, value in zip(fitting_params.keys(), uncertainties)}

# Print uncertainties
print( uncertainties_dict )

In [11]:
# ---------------------
# Save results to files
# ---------------------

# Recompute the eigenvalues of the optimized matrix
A_tensor = matrix_Hamiltonian(best_fitting_params, fixed_params, ratios)
#A_tensor = A_tensor.to('cuda')
optimized_eigenvalues          = pt.linalg.eigvals(A_tensor).real.cpu().detach()
optimized_eigenvalues, indices = pt.sort( optimized_eigenvalues - pt.min(optimized_eigenvalues) )

# Path and file
directory = 'fitting-trials/'
file_name = 'Er_PyTorch'

# Export optimized eigenvalues to be read in Mathematica
optimized_eigenvalues_panda = pd.DataFrame(optimized_eigenvalues)
optimized_eigenvalues_panda.to_csv(directory + file_name + "_optimized_eigenvalues.csv", index=False, header=None)

# Convert target eigenvalues to a pandas DataFrame and Export to be read in Mathematica
target_eigenvalues_panda = pd.DataFrame(target_eigenvalues)
target_eigenvalues_panda.to_csv(directory + file_name + "_target_eigenvalues.csv", index=False, header=None)

# Export trials
with open(directory+file_name + "_optimization_trials.txt", "w") as f: 
    for iteration_data in output_data:
        f.write( str(iteration_data) + "\n" )

# Convert best params from tensor to numerical values
numeric_best_fitting_params = {key: value.detach().item() for (key, value) in fitting_params.items()}
numeric_best_params         = numeric_best_fitting_params | fixed_params | ratios

# Export initial and optimized params
with open(directory + file_name+"_optimized_params.txt", "w") as f:
    f.write(  " Initial Parameters: \n"              + str(numeric_initial_params) \
            + "\n\n Optimized Parameters: \n"        + str(numeric_best_params) \
            + "\n\n Best fitted params: \n"          + str(numeric_best_fitting_params) \
            + "\n\n Uncertainties: \n"               + str(uncertainties_dict) \
            + "\n\n Number of experimental levels: " + str(len(target_index_set)) \
            + "\n Number of fitting parameters: "    + str(len(best_fitting_params)) )
